In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

class PPOMModel(tf.keras.Model):
    def __init__(self, input_shape):
        super(PPOMModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape)
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1)  # Regression output

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

def train_ppom(data, labels, epochs=50, batch_size=32):
    model = PPOMModel(input_shape=(data.shape[1],))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    model.fit(data, labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("R² Score:", r2_score(y_test, y_pred))

def main():
    # Load your dataset
    data = pd.read_csv('patient_outcomes.csv')  # Your dataset path
    labels = data.pop('outcome')  # Continuous labels for outcomes
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data.values, labels.values, test_size=0.2, random_state=42)

    # Standardizing the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    model = train_ppom(X_train, y_train, epochs=50, batch_size=32)

    # Evaluate the model
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    main()